In [16]:
from openai import OpenAI
import time
import json
from nltk import tokenize
from difflib import get_close_matches
from datetime import datetime
from dotenv import load_dotenv
from dotenv import dotenv_values
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import requests
import json
config = dotenv_values(".env")

### Initialize Open AI client

* Create a .env file outside of the Jupyter Notebook

In [2]:
client = OpenAI(api_key=config['OPENAI_KEY'])

### Load in our data on .Gov Domains

In [3]:
gov_domains = pd.read_csv('../R Scripts/Government Domains/Government Domains with WHOIS Data from Cybersecurity & Infrastructure Security Agency.csv')
gov_domains['Agency'].value_counts()[0:10] #Number of websites associated to each agency...showing top 10

Agency
Department of Health and Human Services    131
General Services Administration            106
Department of the Treasury                  94
Department of Justice                       79
Department of the Interior                  67
Department of Commerce                      66
Department of Energy                        65
Department of Homeland Security             44
Executive Office of the President           39
Department of Defense                       39
Name: count, dtype: int64

In [4]:
gov_domains.head()

,Domain.name,Domain.type,Agency,Organization.name,City,State,Security.contact.email,Domain.Name,Registrar.WHOIS.Server,Registrar.URL,...,Security.Postal.Code,Security.Country,Security.Phone,Security.Email,Name.Server,DNSSEC,Registrant.Fax,Admin.Fax,president,simple_date
0,achp.gov,Federal - Executive,Advisory Council on Historic Preservation,Advisory Council on Historic Preservation,Washington,DC,domainsecurity@achp.gov,achp.gov,whois.nic.gov,https://get.gov,...,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,domainsecurity@achp.gov,ns-605.awsdns-11.net,signedDelegation,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,Bill Clinton,1997-10-02
1,arc.gov,Federal - Executive,Appalachian Regional Commission,Appalachian Regional Commission,Washington,DC,(blank),arc.gov,whois.nic.gov,https://get.gov,...,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,karl.ns.cloudflare.com,unsigned,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,Bill Clinton,1997-10-02
2,asc.gov,Federal - Executive,Appraisal Subcommittee of the Federal Financia...,Appraisal Subcommittee,Washington,DC,(blank),asc.gov,whois.nic.gov,https://get.gov,...,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,edns4.ultradns.org,signedDelegation,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,Bill Clinton,1997-10-02
3,aftac.gov,Federal - Executive,Department of Defense,Air Force Technical Applications Center,Patrick AFB,FL,dco@aftac.gov,aftac.gov,whois.nic.gov,https://get.gov,...,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,dco@aftac.gov,dstork.aftac.gov,unsigned,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,Bill Clinton,1997-10-02
4,ameslab.gov,Federal - Executive,Department of Energy,Ames Laboratory,Ames,IA,cybersec@ameslab.gov,ameslab.gov,whois.nic.gov,https://get.gov,...,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,cybersec@ameslab.gov,scsds.ameslab.gov,signedDelegation,NaN,NaN,Bill Clinton,1997-10-02


### Create a base message prompt 

* The objective is to iterate through the .gov domains we collected and extract information from them so that we can analyze similarity between website and eliminate those that are no longer active. This will be part of the data augmentation and cleaning.

In [6]:
def process_response(record_data):
    messages = []
    messages.append( 
        {
        "role": "system",
        "content": '''You are a helpful assistant that is assisting in analyzing the efficiency of the structure of government.
        You will be given a domain name, html associated with the domain, domain type, parent agency,
        organization name, city, state, date of creation of the website, and a last updated domain date. 
        If the website html is "No information found" assume that the organization is no longer active.
        If the website html points to another agency that original oranization name may be replaced by a new organization name.
        We want to get some key information for the organization and we want to create a set of keywords that can later be used to form a similarity matrix between organizations.
        You can use your own knowledge along with the information the user provides to generate your response. 
        Please return in a JSON format the following information:
        {
        original_organization_name: str,
        new_organization_name: str,
        organization_status: ['Active','Not Active'],
        summary_of_organization: str,
        ten_keywords_describing_organization: [str],
        estimated_funding:str,
        source_estimated_funding:str,
        estimated_number_of_employees:str,
        source_estimated_number_of_employees:str,
        }
        '''}, 
    )
    messages.append({
        'role':'user',
        'content':str(record_data)
    })
    chat = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        response_format={ "type": "json_object" } 
    ) 
    reply = chat.choices[0].message.content
    return reply

### Given the above prompt we will extract only the core columns we plan to feed into ChatGPT

In [7]:
records = gov_domains[['Domain.name','Domain.type','Agency','Organization.name','City','State','Creation.Date','Updated.Date']].to_dict(orient='records')

### Now we will iterate through each domain, we will use requests and BS4 to scrape the text from the webpage. The webpage information will be combined with the above records and passed to ChatGPT. We will store the results in a list that we can later furhter clean and merge into the original data. 

In [8]:
store_responses = []

In [15]:
for i in range(926,len(records)):
    #Generate the https:// url to collect the webpage data
    url = 'https://'+gov_domains['Domain.name'].values[i]
    #Try to read the data
    try:
        #Max timeout set to 5-seconds
        response = requests.get(url,timeout=5)
        #read HTML into BS4 object
        soup = BeautifulSoup(response.text, "lxml")
        #focus on body text replacing unnecssary characters to minimze tokens sent to OpenAI
        website_text = soup.body.text.replace('\n',' ').replace('  ','').replace('\t','')
    except:
        #If no text found just define this as no information found for the domain
        website_text = 'No information found'
    #Merge in the data into the existing record data we have
    records[i]['html'] = website_text
    #Feed the combined data into chatgpt and store the response
    store_responses.append(process_response(records[i]))
    #Pause for an average of 6.5 seconds (random uniform)
    time.sleep(np.random.uniform(3,10,1)[0])
    #Print status
    print(i/len(records))

0.6759124087591241
0.6766423357664234
0.6773722627737226
0.6781021897810219
0.6788321167883211
0.6795620437956205
0.6802919708029197
0.6810218978102189
0.6817518248175183
0.6824817518248175
0.6832116788321168
0.6839416058394161
0.6846715328467153
0.6854014598540146
0.6861313868613139
0.6868613138686132
0.6875912408759124
0.6883211678832116
0.689051094890511
0.6897810218978102
0.6905109489051094
0.6912408759124088
0.691970802919708
0.6927007299270073
0.6934306569343066
0.6941605839416058
0.6948905109489051
0.6956204379562044
0.6963503649635037
0.6970802919708029
0.6978102189781021
0.6985401459854015
0.6992700729927007
0.7
0.7007299270072993
0.7014598540145985
0.7021897810218978
0.7029197080291971
0.7036496350364964
0.7043795620437956
0.7051094890510949
0.7058394160583942
0.7065693430656934
0.7072992700729926
0.708029197080292
0.7087591240875912
0.7094890510948905
0.7102189781021898
0.710948905109489
0.7116788321167883
0.7124087591240876
0.7131386861313869
0.7138686131386861
0.7145985401

/var/folders/ft/rx50676x74s1ly72w4qhj7yh0000gn/T/ipykernel_98659/4273876478.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(response.text, "lxml")


0.8496350364963504
0.8503649635036497
0.8510948905109489
0.8518248175182481
0.8525547445255475
0.8532846715328467
0.8540145985401459
0.8547445255474453
0.8554744525547445
0.8562043795620438
0.8569343065693431
0.8576642335766423
0.8583941605839416
0.8591240875912409
0.8598540145985402
0.8605839416058394
0.8613138686131386
0.862043795620438
0.8627737226277372
0.8635036496350365
0.8642335766423358
0.864963503649635
0.8656934306569343
0.8664233576642336
0.8671532846715329
0.8678832116788321
0.8686131386861314
0.8693430656934307
0.8700729927007299
0.8708029197080291
0.8715328467153285
0.8722627737226277
0.872992700729927
0.8737226277372263
0.8744525547445255
0.8751824817518248
0.8759124087591241
0.8766423357664234
0.8773722627737226
0.878102189781022
0.8788321167883212
0.8795620437956204
0.8802919708029197
0.881021897810219
0.8817518248175182
0.8824817518248175
0.8832116788321168
0.8839416058394161
0.8846715328467153
0.8854014598540146
0.8861313868613139
0.8868613138686131
0.887591240875912

In [17]:
collected_information = [json.loads(x) for x in store_responses]

In [19]:
collected_information_df = pd.DataFrame(collected_information)

In [20]:
collected_information_df.to_csv('Government Domains Data Enhanced with ChatGPT Data.csv')